
<br>
FEniCS tutorial demo program: Poisson equation with Dirichlet conditions.<br>
Test problem is chosen to give an exact solution at all nodes of the mesh.<br>
  -Laplace(u) = f    in the unit square<br>
            u = u_D  on the boundary<br>


In [ ]:
from __future__ import print_function
from fenics import *
import matplotlib.pyplot as plt
import numpy as np

Create mesh and define function space

In [ ]:
mesh = UnitSquareMesh(16, 16)
V = FunctionSpace(mesh, 'P', 2)

In [ ]:
mu = Constant(1.0)
beta0 = 10.0
beta1 = 100.0
beta = Constant((beta0,beta1))

Define boundary condition

In [ ]:
u_D = Expression('(exp(beta0/mu*x[0])-1)/(exp(beta0/mu)-1)*(exp(beta1/mu*x[1])-1)/(exp(beta1/mu)-1)', \
  degree=3, beta0=beta0, beta1=beta1, mu=mu)

In [ ]:
uex = Expression('(exp(beta0/mu*x[0])-1)/(exp(beta0/mu)-1)*(exp(beta1/mu*x[1])-1)/(exp(beta1/mu)-1)', \
  degree=3, beta0=beta0, beta1=beta1, mu=mu)

In [ ]:
dxuex = Expression('beta0*(exp(beta0/mu*x[0]))/(exp(beta0/mu)-1)*(exp(beta1/mu*x[1])-1)/(exp(beta1/mu)-1)', \
  degree=3, beta0=beta0, beta1=beta1, mu=mu)
dyuex = Expression('beta1*(exp(beta0/mu*x[0])-1)/(exp(beta0/mu)-1)*(exp(beta1/mu*x[1]))/(exp(beta1/mu)-1)', \
  degree=3, beta0=beta0, beta1=beta1, mu=mu)

In [ ]:
def boundary(x, on_boundary):
    return on_boundary

In [ ]:
bc = DirichletBC(V, u_D, boundary)

Define variational problem

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(0.0)
#mu = Constant(1.0)
#beta0 = 10.0
#beta1 = 1000.0
#beta = Constant((10.0,1000.0))

In [ ]:
normb = np.sqrt(beta0*beta0+beta1*beta1)
h = CellDiameter(mesh)
print(h)
delta = 1.0
tau = delta*h/normb

In [ ]:
sigma =  Constant(0.0)
a = mu*dot(nabla_grad(u), nabla_grad(v))*dx + dot(beta,nabla_grad(u))*v*dx+sigma*u*v*dx + \
    tau*(-mu*div(nabla_grad(u))+dot(beta,nabla_grad(u))+sigma*u)*\
        (-mu*div(nabla_grad(v))+dot(beta,nabla_grad(v))+sigma*v)*dx
L = f*v*dx + tau*f*(-mu*div(nabla_grad(v))+dot(beta,nabla_grad(v))+sigma*v)*dx
#equation = inner(nabla_grad(u), nabla_grad(v))*dx == f*v*dx

rm = parameters["krylov_solver"] # short form<br>
rm["absolute_tolerance"] = 1E-10<br>
rm["relative_tolerance"] = 1E-6<br>
rm["maximum_iterations"] = 1000<br>
et_log_level(DEBUG)

In [ ]:
set_log_level(LogLevel.ERROR)

Compute solution

In [ ]:
u = Function(V)
solve(a==L, u, bc)#, solver_parameters={"linear_solver": "cg", "preconditioner": "ilu"})
info(parameters,True)
# Plot solution and mesh
#plot(u)
#plot(mesh)
#print('Mesh')
#print(str(mesh))

Save solution to file in VTK format

In [ ]:
vtkfile = File('adrGALS/solution.pvd')
vtkfile << u

Compute error in L2 norm

In [ ]:
error_L2 = errornorm(u_D, u, 'L2')

In [ ]:
err = uex - u
error_L2 = np.sqrt(assemble(err**2*dx(mesh)))
gerrx = grad(u)[0]-dxuex
gerry = grad(u)[1]-dyuex
error_H1 = np.sqrt(assemble((gerrx**2+gerry**2)*dx(mesh)))

Compute maximum error at vertices

In [ ]:
vertex_values_u_D = u_D.compute_vertex_values(mesh)
vertex_values_u = u.compute_vertex_values(mesh)

In [ ]:
error_max = np.max(np.abs(vertex_values_u_D - vertex_values_u))


<br>
coor = mesh.coordinates()<br>
if mesh.num_vertices() == len(vertex_values_u):<br>
  for i in range(mesh.num_vertices()):<br>
    print("u(",coor[i][0],",", coor[i][1], ") =" , vertex_values_u[i])<br>


Print errors

In [ ]:
print('error_L2  =', error_L2)
print('error_H1  =', error_H1)
print('error_max =', error_max)
# Hold plot
#plt.show()